# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor

In [5]:
df = pd.read_csv('previsao_de_renda.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [7]:
#Antes de iniciar os exercícios, vamos excluir as colunas 'Unnamed: 0', 'data_ref' e 'id_cliente', pois elas irão apenas atrapalhar nossa análise.
colunas_deletar = ['Unnamed: 0', 'data_ref', 'id_cliente']
df = df.drop(columns= colunas_deletar)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


In [8]:
df_encoded = pd.get_dummies(df, columns=['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], drop_first=True)

In [9]:
df_encoded.dtypes

posse_de_veiculo                    bool
posse_de_imovel                     bool
qtd_filhos                         int64
idade                              int64
tempo_emprego                    float64
qt_pessoas_residencia            float64
renda                            float64
sexo_M                              bool
tipo_renda_Bolsista                 bool
tipo_renda_Empresário               bool
tipo_renda_Pensionista              bool
tipo_renda_Servidor público         bool
educacao_Pós graduação              bool
educacao_Secundário                 bool
educacao_Superior completo          bool
educacao_Superior incompleto        bool
estado_civil_Separado               bool
estado_civil_Solteiro               bool
estado_civil_União                  bool
estado_civil_Viúvo                  bool
tipo_residencia_Casa                bool
tipo_residencia_Com os pais         bool
tipo_residencia_Comunitário         bool
tipo_residencia_Estúdio             bool
tipo_residencia_

In [10]:
df_encoded.columns

Index(['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda', 'sexo_M',
       'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor público',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União',
       'estado_civil_Viúvo', 'tipo_residencia_Casa',
       'tipo_residencia_Com os pais', 'tipo_residencia_Comunitário',
       'tipo_residencia_Estúdio', 'tipo_residencia_Governamental'],
      dtype='object')

In [11]:
df_encoded[['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
         'sexo_M',
       'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor público',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União',
       'estado_civil_Viúvo', 'tipo_residencia_Casa',
       'tipo_residencia_Com os pais', 'tipo_residencia_Comunitário',
       'tipo_residencia_Estúdio', 'tipo_residencia_Governamental']] = df_encoded[['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
         'sexo_M',
       'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor público',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União',
       'estado_civil_Viúvo', 'tipo_residencia_Casa',
       'tipo_residencia_Com os pais', 'tipo_residencia_Comunitário',
       'tipo_residencia_Estúdio', 'tipo_residencia_Governamental']].astype(int)

In [12]:
df_encoded.dtypes

posse_de_veiculo                   int32
posse_de_imovel                    int32
qtd_filhos                         int32
idade                              int32
tempo_emprego                    float64
qt_pessoas_residencia            float64
renda                            float64
sexo_M                             int32
tipo_renda_Bolsista                int32
tipo_renda_Empresário              int32
tipo_renda_Pensionista             int32
tipo_renda_Servidor público        int32
educacao_Pós graduação             int32
educacao_Secundário                int32
educacao_Superior completo         int32
educacao_Superior incompleto       int32
estado_civil_Separado              int32
estado_civil_Solteiro              int32
estado_civil_União                 int32
estado_civil_Viúvo                 int32
tipo_residencia_Casa               int32
tipo_residencia_Com os pais        int32
tipo_residencia_Comunitário        int32
tipo_residencia_Estúdio            int32
tipo_residencia_

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [33]:
#1.
X = df_encoded.drop(columns= ['renda']).fillna(method='bfill').copy()
y = df_encoded['renda']

C:\Users\samsun\AppData\Local\Temp\ipykernel_19048\117927091.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = df_encoded.drop(columns= ['renda']).fillna(method='bfill').copy()


In [34]:
X_train, X_teste, y_train, y_teste = train_test_split(X,y, test_size = 0.25)

In [31]:
#2. Neste caso usaremos o método 'elastic_net', sendo que iremos variar apenas o valor de L1_wt para alternar entre ridge e lasso.
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r_square_ridge = []
modelo = 'renda ~  C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment("Assalariado")) + C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia'

for alpha in alphas:
    # Criar e ajustar um modelo Ridge
    ridge_reg = Ridge(alpha=alpha)
    ridge_reg.fit(X_train, y_train)
    
    # Fazer previsões no conjunto de teste
    y_pred = ridge_reg.predict(X_teste)
    
    # Calcular o R-quadrado
    r2 = r2_score(y_teste, y_pred)
    r_square_ridge.append(r2)
    

In [35]:
r_square_ridge

[0.21576937834333187,
 0.21576938563191328,
 0.2157694147508329,
 0.21576945106995837,
 0.21576973846864944,
 0.2157700899583742]

In [17]:
max(r_square_ridge)

0.22786322868134923

O maior valor de r-quadrado neste modelo é obtido com alpha = 0.1.

In [20]:
#3. Faremos o mesmo processo, mas agora com o método LASSO
r_square_lasso = []

for alpha in alphas:
    md = smf.ols(modelo, data= df)
    reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = alpha)
    r_square_lasso.append(reg.rsquared)

In [21]:
r_square_lasso

[0.25622676364926733,
 0.25622676364926733,
 0.25622676364926733,
 0.25622676364926733,
 0.2562206336762455,
 0.2562206336762455]

In [22]:
max(r_square_lasso)

0.25622676364926733

Neste caso, os maiores valores foram obtidos com alpha sendo 0, 0.001, 0.005 e 0.01, todos os valores de r-quadrado neste caso são iguais

In [23]:
#4.
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    """
    Realiza a seleção de variáveis stepwise para um modelo de regressão linear.
    
    Parâmetros:
    - X: DataFrame com as variáveis independentes.
    - y: Série ou array com a variável dependente.
    - initial_list: Lista inicial de variáveis a serem incluídas no modelo.
    - threshold_in: Valor-p máximo para incluir uma variável.
    - threshold_out: Valor-p mínimo para remover uma variável.
    - verbose: Se True, imprime detalhes durante o processo.
    
    Retorna:
    - Modelo ajustado após a seleção stepwise.
    """
    
    included = list(initial_list)
    while True:
        changed = False
        
        # Adicionar variáveis
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionada variável "{best_feature}" com p-valor {best_pval:.4f}')
        
        # Remover variáveis
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f'Removida variável "{worst_feature}" com p-valor {worst_pval:.4f}')
        
        if not changed:
            break
    
    return sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()

# Exemplo de uso
# Suponha que X seja seu DataFrame de variáveis independentes e y seja a variável dependente
modelo_final = stepwise_selection(X_teste, y_teste)

# Exiba um resumo do modelo final
print(modelo_final.summary())


Adicionada variável "tempo_emprego" com p-valor 0.0000
Adicionada variável "sexo_M" com p-valor 0.0000
Adicionada variável "tipo_renda_Pensionista" com p-valor 0.0031
Adicionada variável "idade" com p-valor 0.0001
Adicionada variável "qt_pessoas_residencia" com p-valor 0.0022
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     216.6
Date:                Wed, 17 Jan 2024   Prob (F-statistic):          1.65e-203
Time:                        16:01:07   Log-Likelihood:                -39044.
No. Observations:                3750   AIC:                         7.810e+04
Df Residuals:                    3744   BIC:                         7.814e+04
Df Model:                           5                                         
Covariance T

5. Comparando os modelos pelos valores de r-quadrado, aquele que possui os maiores valores, logo é o melhor de todos, é o modelo feito pelo método de LASSO. 

In [24]:
#6. Usaremos agora os mesmo modelos tratados no exercício anterior deste módulo, veremos qual se sai melhor.
novo_modelo = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego '
r_square_lasso_2 = []

for alpha in alphas:
    md = smf.ols(novo_modelo, data= df)
    reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = alpha)
    r_square_lasso_2.append(reg.rsquared)

In [25]:
r_square_lasso_2

[0.34652172503586076,
 0.34397654165643443,
 0.34299202325971534,
 0.34522716413463206,
 0.34299202325971534,
 0.34299202325971534]

In [26]:
modelo_3 = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment("Assalariado")) + C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'
r_square_lasso_3 = []

for alpha in alphas:
    md = smf.ols(modelo_3, data= df)
    reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = alpha)
    r_square_lasso_3.append(reg.rsquared)

In [107]:
r_square_lasso_3

[0.357470405817387,
 0.35410210770032846,
 0.3458224190806556,
 0.3456857611205141,
 0.34299202325971534,
 0.34299202325971534]

Ao compararmos os dois modelos usados na lista de exercícios anterior, vemos que o mesmo tratamento aumentou significantemente o valor de r-quadrado, o terceiro modelo porém, se comparado ao segundo, teve uma aumento insignificante comparado ao aumento entre o segundo modelo e o primeiro, além disso, a grande quantidade de variáveis aumenta consideravelmente o risco de overfitting, por isso, o melhor modelo é o segundo.

In [27]:
arvore = DecisionTreeRegressor(random_state=0)
arvore.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [28]:
previsao = arvore.predict(X_teste)
r2_score(y_teste, previsao)

0.37896838698232327

O Valor de r-quadrado desta árvore é maior que qualquer valor obtido anteriormente (com exceção do modelo com overfitting), porém, a diferença é bastante pequena, seria necessárias outras análises para afirmar se essa diferença é insignificante ou não.